## AEX - Arbitrum Ecosystem Index - Methodology Template

## Configuring packages

In [5]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyBase
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [6]:
min_mcap =  0.5e8
min_weight = 0.005
max_weight = 0.3
circ_supply_threshold = 0.2
liveness_threshold = 180 # In days
liquidity_consistency = 90 # In days
coingecko_category = ['arbitrum-ecosystem']

# Calculating max slippage based on liquidity threshold
liq = 5*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = -0.09

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = MethodologyBase(
                                'arbitrum-one',
                                min_mcap,
                                min_weight,
                                max_weight,
                                255,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                max_slippage,
                                100000,
                                coingecko_category)

## Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data spanning at least 6 months prior to the date of inclusion in the index.
    1. The project should have an active community.
    1. No rebasing or deflationary tokens.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project must have a minimum circulating market cap of $50mm.
   <br>
1. **Pricing requirmeents**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 20% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The price should experience no more than 12% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.
1. **Weighting requirements**
    1. The maximum weight any one token can have is 30%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


## Run Methodology

In [7]:

results,liquidity = methodology_class.main(
    single_chain='arbitrum-one',
    df_to_remove=[stablecoins,ctokens,atokens],
    add_category_assets=['joe'],
    onchain_oracles=['radiant-capital'],
    remove_category_assets=['wrapped-bitcoin','rocket-pool-eth','ankreth','bitcoin-avalanche-bridged-btc-b','tbtc','staked-frax-ether','renzo-restaked-eth','wrapped-eeth']
    )
liquidity

Excluding xai-blockchain, pricing data available only for 109 < 180 days
Excluding ox-fun, pricing data available only for 67 < 180 days
sweth on arbitrum-one returned an invalid API response
sweth recorded no valid response across any platform
ankr on arbitrum-one returned an invalid API response
ankr recorded no valid response across any platform
dao-maker on arbitrum-one returned an invalid API response
dao-maker recorded no valid response across any platform
Manually calculating slippage for seedify-fund on arbitrum-one
cartesi on arbitrum-one returned an invalid API response
cartesi recorded no valid response across any platform
orbs on arbitrum-one returned an invalid API response
orbs recorded no valid response across any platform
tellor on arbitrum-one returned an invalid API response
tellor recorded no valid response across any platform
Manually calculating slippage for opulous on arbitrum-one
perpetual-protocol on arbitrum-one returned an invalid API response
perpetual-protoc

,arbitrum-one,optimal chain,best slippage,best slippage chain
id,,,,
arbitrum,-0.003093,arbitrum-one,-0.003093,arbitrum-one
uniswap,-0.006207,arbitrum-one,-0.006207,arbitrum-one
chainlink,-0.008084,arbitrum-one,-0.008084,arbitrum-one
pendle,-0.011604,arbitrum-one,-0.011604,arbitrum-one
gmx,-0.012824,arbitrum-one,-0.012824,arbitrum-one
radiant-capital,-0.014996,arbitrum-one,-0.014996,arbitrum-one
magic,-0.020551,arbitrum-one,-0.020551,arbitrum-one
the-graph,-0.038956,arbitrum-one,-0.038956,arbitrum-one
gains-network,-0.057346,arbitrum-one,-0.057346,arbitrum-one


## Show Results

In [8]:
results

,symbol,market_cap,weight,weight_converted,address,blockchain
id,,,,,,
chainlink,LINK,8626692591,0.300000,77,0xf97f4df75117a78c1a5a0dbb814af92458539fb4,arbitrum-one
uniswap,UNI,5838019201,0.300000,77,0xfa7f8980b0f1e64a2062791cc3b0871572f1f7f0,arbitrum-one
arbitrum,ARB,2867929896,0.174985,45,0x912ce59144191c1204e64559fe8253a0e49e6548,arbitrum-one
the-graph,GRT,2479869598,0.151308,39,0x9623063377ad1b27544c965ccd7342f7ea7e88c7,arbitrum-one
pendle,PENDLE,529245891,0.032292,9,0x0c880f6761f1af8d9aa9c466984b80dab9a8c9e8,arbitrum-one
gmx,GMX,255451868,0.015586,3,0xfc5a1a6eb076a2c7ad06ed22c90d7e710e35ad0a,arbitrum-one
magic,MAGIC,204892741,0.012501,3,0x539bde0d7dbd336b79148aa742883198bbf60342,arbitrum-one
gains-network,GNS,116875153,0.007131,1,0x18c11fd286c5ec11c3b683caa813b77f5163a122,arbitrum-one
radiant-capital,RDNT,101568052,0.006197,1,0x3082cc23568ea640225c2467653db90e9250aaa0,arbitrum-one
